# RAG 시스템 - 뷰티 코칭 AI (Hybrid: Ollama + OpenAI)

이 노트북은 PDF 문서를 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템입니다.

## 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: Ollama llama3 (무료, 로컬)
- **LLM**: OpenAI GPT-4o (유료, 고품질, 비전 지원)

## 주요 기능
- PDF 문서 로드 및 전처리
- Ollama llama3 임베딩 및 Chroma DB 저장 (무료!)
- OpenAI GPT-4o 비전 모델을 사용한 이미지+텍스트 질의응답

## 1. 라이브러리 Import

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import re
import json
import time
import os
import base64
from pathlib import Path

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")

## 2. 유틸리티 함수 정의

In [ ]:
def clean_text(text):
    """텍스트 공백 정리"""
    text = re.sub(r' {2,}', ' ', text)
    text = text.replace('\t', ' ')
    text = re.sub(r'\n{3,}', '\n\n', text)
    lines = [line.strip() for line in text.split('\n')]
    text = '\n'.join(lines)
    lines = [line for line in text.split('\n') if line]
    text = '\n'.join(lines)
    return text.strip()

def load_documents():
    """PDF 문서 로드"""
    loader = DirectoryLoader(
        "hairdata/",
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
    return documents

def chunk_doc(documents, chunk_size=500, chunk_overlap=50):
    """문서를 청크로 분할"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
        keep_separator=True
    )
    splits = text_splitter.split_documents(documents)
    return splits

def save_documents(documents, output_path):
    """문서를 JSONL 형식으로 저장"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, page in enumerate(documents):
            data = {
                'page': i + 1,
                'text': page.page_content,
                'metadata': {
                    'source': 'sample.pdf',
                    'page': page.metadata.get('page', i)
                }
            }
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

def encode_image(image_path):
    """이미지를 base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

print("✅ 유틸리티 함수 정의 완료")

## 3. 문서 로드 및 전처리

**이 셀은 한 번만 실행하세요!**

In [ ]:
print("문서 로드 중...")
documents = load_documents()
print(f"✅ 총 {len(documents)}개의 문서 로드 완료")

print("\n문서 청킹 중...")
chunked_documents = chunk_doc(documents)
print(f"✅ 총 {len(chunked_documents)}개의 청크 생성 완료")

print("\n문서 저장 중...")
save_documents(chunked_documents, 'tmp.jsonl')
print("✅ tmp.jsonl 저장 완료")

## 4. 벡터 임베딩 및 저장 (Ollama llama3)

**이 셀도 한 번만 실행하세요! 이미 임베딩이 있다면 5번 셀로 바로 가세요.**

**Ollama를 사용하므로 무료입니다!** ⭐

In [ ]:
print("=" * 50)
print("Ollama llama3 임베딩 시작...")
print(f"청크 개수: {len(chunked_documents)}개")
print("⭐ Ollama 로컬 모델 사용 - 무료!")
print("=" * 50)

embedding_start = time.time()

# Ollama llama3 임베딩 모델 사용 (로컬, 무료)
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3"
)

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
    persist_directory="./chroma_db_hybrid"
)

embedding_end = time.time()
print(f"✅ Ollama llama3 임베딩 완료: {embedding_end - embedding_start:.2f}초")
print("=" * 50)

## 5. 기존 벡터스토어 로드 (Ollama llama3)

**임베딩을 다시 하지 않고 기존 것을 사용하려면 이 셀을 실행하세요!**

**Ollama 로컬 모델이므로 무료입니다!** ⭐

In [ ]:
if os.path.exists("./chroma_db_hybrid"):
    print("기존 Ollama llama3 벡터스토어 로드 중...")
    from langchain_ollama import OllamaEmbeddings

    embeddings = OllamaEmbeddings(
        model="llama3"
    )
    vectorstore = Chroma(
        persist_directory="./chroma_db_hybrid",
        embedding_function=embeddings
    )
    print("✅ 벡터스토어 로드 완료")
else:
    print("❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.")

## 6. RAG 체인 설정 (Hybrid: Ollama 임베딩 + OpenAI GPT-4o)

**비용 효율적인 하이브리드 접근:**
- **임베딩**: Ollama llama3 (무료, 로컬)
- **LLM/Chat**: OpenAI GPT-4o (유료, 고품질, 비전 지원)

In [ ]:
# Retriever 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# OpenAI GPT-4o 모델 설정 (비전 지원)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

print(f"✅ 사용 모델: gpt-4o (비전 지원)")

# LCEL 방식으로 체인 구성
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_message_with_image(inputs):
    """이미지와 텍스트를 함께 메시지로 생성"""
    context = inputs["context"]
    question = inputs["question"]
    image_data = inputs.get("image")

    # 프롬프트 텍스트 구성 (한글 답변 강제)
    prompt_text = f"""당신은 다음 3명의 전문가가 통합된 고급 이미지·뷰티 코칭 AI입니다.
- 1. 헤어 스타일리스트: 헤어 형태, 볼륨, 질감, 스타일 방향성, 머릿결 관리 등 분석
- 2. 피부 관리 전문가: 피부 톤, 수분·유분 상태, 결, 잡티, 광택 등 관찰
- 3. 얼굴 윤곽 디자이너: 눈·코·입 비율, 턱선, 볼살, 이마 곡선, 대칭, 입체감 등 분석

[분석 방식 지침]
1. 사용자가 제공한 성별, 직업, 상황 설명을 기반으로 한다.
2. 관찰은 반드시 시각적·감각적 요소 중 최소 3가지 이상을 활용하고 디테일을 풍부하게 묘사하여, 독자가 장면을 그릴 수 있게 작성한다.
3. 부정적인 표현을 피하고 개선 가능성을 중심으로 서술한다.
4. 개선 팁은 오늘 실천 가능한 구체적 행동이나 셀프 케어 방법만 포함해야 하며, 추상적 조언은 금지된다.
5. 따뜻하고 격려적인 어조를 유지하며, 객관적이되 전문적으로 작성한다.
6. 전문 용어를 적절히 사용하되 이해하기 쉬운 표현으로 설명한다.

[분석 카테고리]
1. 헤어(Hair)
2. 피부(Skin)
3. 윤곽(Contour)

[Output JSON Example]
{{{{
  "Appearance_Coaching": {{{{
    "Hair": {{{{
      "status": "헤어의 현재 상태에 대한 간결한 요약 설명",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}},
    "Skin": {{{{
      "status": "피부의 현재 상태에 대한 간결한 요약 설명",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}},
    "Contour": {{{{
      "status": "윤곽의 현재 상태에 대한 간결한 요약 설명",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}}
  }}}}
}}}}

**중요: 답변은 반드시 한글로만 작성해야 합니다. 영어는 절대 사용하지 마세요.**
출력은 반드시 위 JSON 형식으로 하며, 개선 팁은 반드시 오늘 실천 가능한 구체적 행동으로 작성하세요.

[참고 문서]
{context}

질문: {question}

답변:"""

    # 이미지가 있는 경우 멀티모달 메시지 생성
    if image_data:
        return [HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
            ]
        )]
    else:
        # 이미지가 없는 경우 텍스트만 전달
        return [HumanMessage(content=prompt_text)]

qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["question"])),
        "question": lambda x: x["question"],
        "image": lambda x: x.get("image")
    }
    | RunnableLambda(create_message_with_image)
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인 설정 완료 (이미지 지원)")

## 7. 질의응답 실행 (텍스트만)

**이미지 없이 텍스트만 사용하는 경우**

In [ ]:
# 질문을 여기서 수정하세요
query = "화사해보이고 싶은데 염색 추천해줘"

print("=" * 50)
print(f"질문: {query}")
print("=" * 50)
print("\nRAG 쿼리 시작... (OpenAI API 호출 중)")
invoke_start = time.time()

result = qa_chain.invoke({"question": query})

invoke_end = time.time()
print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
print("=" * 50)

print("\n📝 답변:")
print(result)

## 8. 질의응답 실행 (이미지 포함)

**이미지와 함께 질문하는 경우**

**주의: GPT-4o 비전 모델은 비용이 더 많이 발생합니다!**

In [ ]:
# 이미지 경로와 질문을 수정하세요
image_path = "sample_image.jpg"
query = "이 사진을 분석해서 어울리는 염색 추천해줘"

if Path(image_path).exists():
    print("=" * 50)
    print(f"이미지: {image_path}")
    print(f"질문: {query}")
    print("=" * 50)
    print("\n이미지 인코딩 중...")
    
    image_base64 = encode_image(image_path)
    print("✅ 이미지 인코딩 완료")
    
    print("\nRAG 쿼리 시작... (OpenAI GPT-4o Vision API 호출 중)")
    invoke_start = time.time()
    
    result = qa_chain.invoke({
        "question": query,
        "image": image_base64
    })
    
    invoke_end = time.time()
    print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
    
    print("\n📝 답변:")
    print(result)
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")

## 9. 참고 문서 확인 (선택사항)

In [ ]:
# 마지막 질문에 대한 참고 문서 확인
retrieved_docs = retriever.invoke(query)

print("📚 참고 문서:")
print("=" * 50)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    print(doc.page_content)
    print("-" * 50)

## 사용 방법 요약

### 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: Ollama llama3 (무료, 로컬)
- **LLM**: OpenAI GPT-4o (유료, 고품질, 비전 지원)

### 처음 실행할 때:
1. **.env 파일에 OpenAI API 키 설정** (OPENAI_API_KEY=sk-...)
2. **Ollama llama3 모델 설치 확인** (터미널에서 `ollama pull llama3`)
3. 셀 1-4를 순서대로 실행 (Ollama 임베딩 생성) ⭐ **무료!**
4. 셀 6을 실행 (RAG 체인 설정)
5. 셀 7을 실행 (텍스트만) 또는 셀 8을 실행 (이미지 포함) ⚠️ **OpenAI API 비용 발생**

### 다음번에 실행할 때 (임베딩 재사용):
1. 셀 1-2를 실행 (라이브러리 및 함수 로드)
2. **셀 5를 실행** (기존 벡터스토어 로드) ⭐ **4번 대신 5번!**
3. 셀 6을 실행 (RAG 체인 설정)
4. 셀 7 (텍스트만) 또는 셀 8 (이미지 포함) 실행

### 비용 및 성능 팁:
- **임베딩 (Ollama)**: 완전 무료! 로컬에서 실행됨
- **텍스트 질의 (OpenAI)**: GPT-4o API 비용만 발생, 비교적 저렴
- **이미지 질의 (OpenAI)**: GPT-4o 비전 모델, 비용이 더 높지만 이미지 분석 가능
- **한글 답변**: 프롬프트에 한글 강제 지시문 추가됨
- 셀 7 또는 8의 `query` 변수만 수정해서 여러 질문을 테스트하세요.

### 기존 OpenAI 임베딩에서 마이그레이션:
- 기존에 `./chroma_db_openai` 디렉토리가 있다면, 셀 4를 실행해서 새로운 Ollama 임베딩을 생성하세요.
- 새로운 임베딩은 `./chroma_db_hybrid` 디렉토리에 저장됩니다.